In [1]:
# Imports for Gemini (LangChain) in the same structure as OpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain  # keep if you still use it elsewhere
from langchain_core.output_parsers import StrOutputParser
from langchain.callbacks import StdOutCallbackHandler
# Note: LLMChain is deprecated; use Prompt | llm | StrOutputParser instead.

import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2

# Load environment variables from .env (expects GOOGLE_API_KEY)
load_dotenv()


True

In [3]:
KEY = os.getenv("GOOGLE_API_KEY")

In [8]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.6, google_api_key=KEY)
print(llm.invoke("Reply with OK").content)

OK


In [5]:
llm

ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), temperature=0.6, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x00000283B80A3380>, default_metadata=())

In [9]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [6]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [7]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [39]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [40]:
TEMPLATE="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [41]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [42]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [43]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [44]:

"""pdf_reader=PyPDF2.PdfReader(r"C:\Complete_Content\All_Project\TEST_FOR_EVERYTHING\langchain\experiments\Big Data Assessment.pdf")
text = ""
for page in pdf_reader.pages:
    text += page.extract_text()"""

<>:1: SyntaxWarning: invalid escape sequence '\C'
<>:1: SyntaxWarning: invalid escape sequence '\C'
C:\Users\Rajat Pandey\AppData\Local\Temp\ipykernel_34732\2136195752.py:1: SyntaxWarning: invalid escape sequence '\C'
  """pdf_reader=PyPDF2.PdfReader(r"C:\Complete_Content\All_Project\TEST_FOR_EVERYTHING\langchain\experiments\Big Data Assessment.pdf")


'pdf_reader=PyPDF2.PdfReader(r"C:\\Complete_Content\\All_Project\\TEST_FOR_EVERYTHING\\langchain\\experiments\\Big Data Assessment.pdf")\ntext = ""\nfor page in pdf_reader.pages:\n    text += page.extract_text()'

In [46]:
file_path = r"C:\Users\Rajat Pandey\Desktop\AI Engineering\mcqgen_0.1\data.txt"

In [47]:
file_path

'C:\\Users\\Rajat Pandey\\Desktop\\AI Engineering\\mcqgen_0.1\\data.txt'

In [48]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [50]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[6][7] The synonym self-teaching computers was also used in this time period.[8][9]

The earliest machine learning program was introduced in the 1950s when Arthur Samuel invented a computer program that calculated the winning chance in checkers for each side, but the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[10] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[11] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[10] Other researchers who have studied human cognitive systems contribu

In [51]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [52]:
"""mcq_count=5
grade=1
tone="simple"""

'mcq_count=5\ngrade=1\ntone="simple'

In [53]:
NUMBER=5 
SUBJECT="data science"
TONE="simple"

In [57]:
from langchain.callbacks import StdOutCallbackHandler

handler = StdOutCallbackHandler()
payload = {
  "text": TEXT, "number": NUMBER, "subject": SUBJECT,
  "tone": TONE, "response_json": json.dumps(RESPONSE_JSON)
}
# for LCEL/chains:
out = generate_evaluate_chain.invoke(payload, config={"callbacks": [handler]})
print(out)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[6][7] The synonym self-teaching computers was also used in this time period.[8][9]

The earliest machine learning program was introduced in the 1950s when Arthur Samuel invented a computer program that calculated the winning chance in checkers for each side, but the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[10] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[11] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons us

In [58]:
# direct llm call (shows where to look)
resp = llm.invoke("Short test")
print(resp.content)
print(resp.response_metadata)               # inspect keys

# common metadata keys (may vary by SDK/version)
usage = resp.response_metadata.get("usage_metadata") or resp.response_metadata.get("usage") or resp.response_metadata.get("token_count")
print("usage:", usage)

# if usage present:
# input_tokens = usage.get("input_token_count")
# output_tokens = usage.get("output_token_count")
# total_tokens = usage.get("total_token_count")

Please provide the test!  I need the questions to be able to give you answers.
{'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}
usage: None


In [60]:
import google.generativeai as genai
genai.configure(api_key=KEY)

# example call (SDK surface can vary; inspect returned object)
r = genai.GenerativeModel("gemini-1.5-flash").generate_content("Hello")
print(r.usage_metadata)   # contains token counts if provided

prompt_token_count: 1
candidates_token_count: 11
total_token_count: 12



In [61]:
from langchain.callbacks.tracers import LangChainTracer
tracer = LangChainTracer(project_name="mcqgen")
out = generate_evaluate_chain.invoke(payload, config={"callbacks": [tracer]})
# Visit LangSmith UI for detailed run and token info



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[6][7] The synonym self-teaching computers was also used in this time period.[8][9]

The earliest machine learning program was introduced in the 1950s when Arthur Samuel invented a computer program that calculated the winning chance in checkers for each side, but the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[10] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[11] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons us

Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=c8a06d5e-206c-4351-9e32-ada2512c3022,id=c8a06d5e-206c-4351-9e32-ada2512c3022; trace=c8a06d5e-206c-4351-9e32-ada2512c3022,id=61fccba7-711e-4dd2-86c7-4cd7ea267295; trace=c8a06d5e-206c-4351-9e32-ada2512c3022,id=744b7996-10d5-4997-ae28-a35eafcaaf39



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert english grammarian and writer. Given a Multiple Choice Quiz for data science students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
```json
{
  "1": {
    "mcq": "Who coined the term 'machine learning'?",
    "options": {
      "a": "Donald Hebb",
      "b": "Alan Turing",
      "c": "Arthur Samuel",
      "d": "Tom M. Mitchell"
    },
    "correct": "c"
  },
  "2": {
    "mcq": "What was the earliest machine learning program mentioned in the text primarily designed to do?",
    "options": {
      "a": "Analyze sonar signals",
      "b": "Recognize handwritten characters",
    

Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=c8a06d5e-206c-4351-9e32-ada2512c3022,id=744b7996-10d5-4997-ae28-a35eafcaaf39; trace=c8a06d5e-206c-4351-9e32-ada2512c3022,id=61fccba7-711e-4dd2-86c7-4cd7ea267295; trace=c8a06d5e-206c-4351-9e32-ada2512c3022,id=176da7f6-c935-4017-bffc-3474760c6a8f; trace=c8a06d5e-206c-4351-9e32-ada2512c3022,id=1d56ff49-ee7e-43d0-a129-bc33481293d7



> Finished chain.

> Finished chain.


Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=c8a06d5e-206c-4351-9e32-ada2512c3022,id=1d56ff49-ee7e-43d0-a129-bc33481293d7; trace=c8a06d5e-206c-4351-9e32-ada2512c3022,id=176da7f6-c935-4017-bffc-3474760c6a8f; trace=c8a06d5e-206c-4351-9e32-ada2512c3022,id=c8a06d5e-206c-4351-9e32-ada2512c3022
